# Brief Métier : Prédiction de Churn Clients avec le Deep Learning
__________________________________________________________________________________________________________________________________________________

## 1. Contexte professionnel
Vous êtes **Data Scientists** dans une ESN (Entreprise de Services Numériques) qui accompagne les opérateurs télécoms dans la réduction de la perte d’abonnés.
Votre nouveau client, **TelcoNova**, souhaite anticiper les départs de ses clients (churn) afin d’orienter ses campagnes de rétention.

Il met à votre disposition un extrait anonymisé de sa base CRM (le jeu Telco Customer Churn, déjà pré-nettoyé en grande partie) et vous laisse **3 jours** pour livrer un premier prototype de modèle de prédiction exploitable en production.

TelcoNova exige un livrable reproductible et facilement intégrable par ses équipes MLOps ; vous travaillerez **en binôme**, en suivant les bonnes pratiques Git / GitHub.


__________________________________________________________________________________________________________________________________________________

## 2. Votre mission

1. **Explorer & préparer les données** :

    + audit qualité, gestion des valeurs manquantes, etc.


In [ ]:
import pandas as pd
import numpy as np

dataframe = pd.read_csv('../data/WA_Fn-UseC_-Telco-Customer-Churn.csv')
for index, column in enumerate ( dataframe.columns) :
    print(f"{index}: {column}, ", end='')
    num_values = dataframe[column].nunique()
    is_numeric = pd.api.types.is_numeric_dtype(dataframe[column])
    numeric_type = str(dataframe[column].dtype)
    print(f"{num_values}{(' '+numeric_type) if is_numeric else ''} values", end='')
    if num_values < 10 :
        print(f": " + ', '.join(str(x) for x in dataframe[column].unique())) 
    else :
        print(f".") 



In [ ]:
# exploration des colonnes à plus de 10 valeurs

def check_float(dataframe : pd.DataFrame, columnname : str) :
    print(f"{columnname}:")
    numericount =0
    for value in dataframe[columnname].unique() :
        try :
            numeric_value = float(value)
            numericount+=1
        except :
            print(f"  [{value}]")

    print(f"{numericount} values")
    print()

# 0: customerID, 7043 values => drop column
# 5: tenure, 73 int64 values => int64
# 18: MonthlyCharges, 1585 float64 values => float64
check_float(dataframe, 'MonthlyCharges')

# 19: TotalCharges, 6531 values : Exploration
check_float(dataframe, 'TotalCharges')


In [ ]:
import sys
import os

# Ajouter le chemin absolu du dossier src au sys.path
sys.path.append(os.path.abspath('../src'))
from pipeline_functions import exclude_spaces

print(f"before : {dataframe['TotalCharges'].dtype}")

dataframe2 = exclude_spaces('TotalCharges')(dataframe)

print(f"after : {dataframe2['TotalCharges'].dtype}")



  + encodage des variables / normalisation / standardisation des variables


In [ ]:
from pipeline_functions import create_preprocessor

preprocessor = create_preprocessor()
preprocessor


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score
from sklearn.model_selection import train_test_split


sklearn_model = LogisticRegression()
sklearn_model.set_params()


complete_pipeline = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", sklearn_model)]
)

y = [1 if x == 'Yes' else 0 for x in dataframe['Churn']]
X = dataframe.drop(columns=['customerID','Churn'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

complete_pipeline.fit(X_train, y_train)

# Prédiction
y_pred = complete_pipeline.predict(X_test)
y_proba = complete_pipeline.predict_proba(X_test)[:, 1]  # pour le ROC AUC

# Évaluation
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)

print("model score: %.3f" % complete_pipeline.score(X_test, y_test))
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"ROC AUC  : {auc:.4f}")





2. **Élaborer un pipeline d’entraînement** sous **TensorFlow Keras ou PyTorch** (au choix) :

  + architecture MLP pensée pour la tabulaire : ≥ 2 couches cachées,

  + fonction de perte adaptée à une classification binaire,

  + gestion éventuelle du déséquilibre de classes.


In [ ]:
# add comment

from pipeline_functions import build_nn_model
import tensorflow as tf

y = [1 if x == 'Yes' else 0 for x in dataframe['Churn']]
X = dataframe.drop(columns=['customerID','Churn'])

X = preprocessor.fit_transform(X)

X_train_0, X_test, y_train_0, y_test = train_test_split( X, y, test_size=0.2, random_state=42, stratify=y )
X_train, X_val, y_train, y_val = train_test_split( X_train_0, y_train_0, test_size=0.2, random_state=42, stratify=y_train_0 )

deepl_classifier = build_nn_model(X_train)
deepl_classifier

  + callbacks (EarlyStopping, ModelCheckpoint, TensorBoard ou équivalent) pour suivre l’entraînement,


In [ ]:
X_train = X_train.toarray() if hasattr(X_train, "toarray") else X_train
X_val = X_val.toarray() if hasattr(X_val, "toarray") else X_val

y_train = np.array(y_train)
y_val = np.array(y_val)

history = deepl_classifier.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=16,
    verbose=1
)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',    # surveille la perte de validation
    patience=3,            # tolère 3 époques sans amélioration
    restore_best_weights=True
)

  + suivi des métriques : ROC-AUC, F1-score, Recall sur la classe Churn = Yes.


In [ ]:
#test_loss, test_acc, test_pre, test_rec, test_auc, test_f1s = deepl_classifier.evaluate(X_val, y_val)
metrics = deepl_classifier.evaluate(X_val,y_val)
test_loss, test_acc, test_pre, test_auc = metrics
print(f"Loss      : {test_loss:.4f}")
print(f"Accuracy  : {test_acc:.4f}")
print(f"Precision : {test_pre:.4f}")
# print(f"Recall    : {test_rec:.4f}")
print(f"ROC-AUC   : {test_auc:.4f}")
#print(f"F1-score  : {test_f1s:.4f}")


In [ ]:
import matplotlib.pyplot as plt

def plot_history(ax, title:str, history, validation=True):
    ax.plot(history.history[title], label=f"Train {title}")

    val_title = "val_"+title
    if validation and val_title in history.history:
        ax.plot(history.history[val_title], label=f"Val {title}")
        
    ax.set_title(f"Évolution de la {title}")
    ax.set_xlabel('Époque')
    ax.set_ylabel(title)
    ax.legend()
    ax.grid()

# Créer la figure et les axes (4 plots côte à côte)
fig1, axs = plt.subplots(1, 4, figsize=(20, 5))

plot_history(axs[0], "loss", history)
plot_history(axs[1], "accuracy", history)
plot_history(axs[2], "precision",history)
plot_history(axs[3], "roc_auc", history)

plt.tight_layout()
plt.show()
    




3. Évaluer, comparer, justifier :

  + fixer un **baseline** simple (régression logistique ou arbre de décision) pour mesurer le gain du réseau,

  + analyser les courbes d’apprentissage, la matrice de confusion, le ROC, les distributions des scores,

  + expliquer (au moins qualitativement) les variables les plus influentes.


4. Exporter le modèle + artefacts :

  + binaire du modèle (<span style="color: lightgreen;"><b>SavedModel</b></span> ↔ TensorFlow / fichier <span style="color: lightgreen;"><b>.pt</b></span> ↔ PyTorch),

  + scaler / encoder,

  + script ou notebook d’inférence qui charge les artefacts et prédit le churn sur de nouvelles données.


__________________________________________________________________________________________________________________________________________________

## 3. Fonctionnalités obligatoires (MVP)


<table style="width:75%">
  <tr>
    <th>Bloc</th>
    <th>Exigences clés</th>
  </tr>
  <tr>
    <th>Data Prep</th>
    <td>
        <ul>
            <li>Nettoyage & typage correct de <b>toutes</b> les colonnes</li>
            <li>Encodage systématique des catégorielle</li>
            <li>Split train / val / test (stratifié)</li>
        </ul>
    </td>
  </tr>
  <tr>
    <th>Modélisation</th>
    <td>
        <ul>
            <li>MLP implémenté <b>from scratch</b> sous TF /Keras <b>ou</b> PyTorch (pas d’API AutoML)</li>
            <li>Backprop + optimiseur (Adam ou SGD)</li>
            <li>Gestion du déséquilibre (pondération ou autre)</li>
        </ul>
    </td>
  </tr>
  <tr>
    <th>Suivi & répétabilité</th>
    <td>
      <ul>
        <li>TensorBoard ou PyTorch Lightning logger </li>
        <li>Seeds fixés + README détaillant la reproduction</li>
        <li>ModelCheckpoint pour restaurer le meilleur modèle</li>
      </ul>
    </td>
  </tr>
  <tr>
    <th>Évaluation</th>
    <td>
      <ul>
        <li>ROC-AUC ≥ 0 .80 <b>OU</b> F1 ≥ 0 .60</li>
        <li>Rapport comparant le réseau au baseline</li>
      </ul>
    </td>
  </tr>
  <tr>
    <th>Collaboration Git</th>
    <td>
      <ul>
        <li>1 branch = 1 feature</li>
        <li> Pull request systématique avec description</li>
      </ul> 
    </td>
  </tr>
</table>





__________________________________________________________________________________________________________________________________________________

## 4. Détails techniques & considérations
  + **Imbalance** : la classe Churn = Yes ≈ 26 %.

  + **Pré-processing dans le même pipeline** que l’inférence (pas de données “fuites” entre train et test).

  + **Hyperparamètres** : batch_size, learning_rate, nb_neurones par couche ; explorez‐les rapidement (Random Search / KerasTuner / Ray Tune) mais documentez votre stratégie.

  + **Éthique & RGPD** : aucune donnée personnellement identifiable ne doit sortir du cadre du projet.


__________________________________________________________________________________________________________________________________________________

## 5. Ressources utiles

  + **Docs officielles** :
 
    + TensorFlow /Keras : https://www.tensorflow.org/api_docs
    + PyTorch : https://pytorch.org/docs/
    + scikit-learn preprocessing : https://scikit-learn.org/stable/modules/preprocessing.html

 + **Outils** :

    + KerasTuner, Optuna (hyper-tuning)
    + Imbalanced-learn (rééchantillonnage)

  + **Cours/vidéos** :

    + [DeepLearning.AI TensorFlow Developer (Coursera) – chapitres 2 & 3](http://DeepLearning.AI)
    + [PyTorch Lightning Crash Course (YouTube)](https://www.youtube.com/watch?v=OIenNRt2bjg)


__________________________________________________________________________________________________________________________________________________

## 6. Livrables
  + Code source python propre et documenté

  + ROC-AUC ≥ 0 .80 ou F1 ≥ 0 .60

  + README clair, pas de données personnelles commitées

  + Présentation prête (10 min max et tous les membres du groupes doivent parler)


____________________________________________________________________________________________________________________________

  + Groupe pour le brief :
    
    G1 = Khadija A.	+ Wael	

    G2 = Victor	+ Maxime + Antoine

    G3 = Ludivine + Dorothée	

    G4 = Malek + Nicolas G	

    G6 = Hacène + Elliandy	

    G7 = Raouf + Léo	

    G8 = Samuel	+ Michael	

    G9 = Sami +	Gauthier	

    G10 = Nicolas C + David	
   

